<a href="https://colab.research.google.com/github/somayeh1404/cod/blob/main/CNN_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


!unzip Chest X-Ray Images (Pneumonia).zip

!ls

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 98% 2.24G/2.29G [00:20<00:01, 31.0MB/s]
100% 2.29G/2.29G [00:20<00:00, 122MB/s] 
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `unzip Chest X-Ray Images (Pneumonia).zip'
chest-xray-pneumonia.zip  sample_data


In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit


BATCH_SIZE = 32
DROPOUT = 0.5
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
IMG_SIZE = 224
PATIENCE = 5


train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def setup_dataset():
    base_path = '/content/chest_xray'
    if not os.path.exists(base_path):
        print("Downloading dataset...")
        os.makedirs('/content', exist_ok=True)
        os.system('kaggle datasets download -d paultimothymooney/chest-xray-pneumonia')
        os.system('unzip -q chest-xray-pneumonia.zip')

    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')

    return train_path, test_path

train_path, test_path = setup_dataset()


train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)


targets = np.array([label for _, label in train_dataset])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)


class_counts = np.bincount(targets[train_idx])
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)


train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(DROPOUT),
    nn.Linear(num_ftrs, 2)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2)


best_val_accuracy = 0
no_improve = 0

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total


    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    scheduler.step(val_acc)

    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%')


    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f'\nEarly stopping at epoch {epoch+1}')
            break


model.load_state_dict(torch.load('best_model.pth'))
model.eval()

all_labels = []
all_preds = []
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_acc = 100 * test_correct / len(test_dataset)
print(f'\nTest Accuracy: {test_acc:.2f}%')
print('\nClassification Report:')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

print('\nConfusion Matrix:')
print(confusion_matrix(all_labels, all_preds))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]



Epoch 1/10:
Train Loss: 0.1995 | Acc: 92.79%
Val Loss: 0.1657 | Acc: 90.52%

Epoch 2/10:
Train Loss: 0.1158 | Acc: 95.30%
Val Loss: 0.1771 | Acc: 95.40%

Epoch 3/10:
Train Loss: 0.1215 | Acc: 95.33%
Val Loss: 0.0740 | Acc: 97.89%

Epoch 4/10:
Train Loss: 0.1053 | Acc: 96.33%
Val Loss: 0.1166 | Acc: 94.64%

Epoch 5/10:
Train Loss: 0.1017 | Acc: 96.45%
Val Loss: 0.0740 | Acc: 96.17%

Epoch 6/10:
Train Loss: 0.0947 | Acc: 96.50%
Val Loss: 0.0675 | Acc: 96.93%

Epoch 7/10:
Train Loss: 0.0632 | Acc: 97.70%
Val Loss: 0.0456 | Acc: 98.47%

Epoch 8/10:
Train Loss: 0.0506 | Acc: 98.11%
Val Loss: 0.0408 | Acc: 98.37%

Epoch 9/10:
Train Loss: 0.0415 | Acc: 98.35%
Val Loss: 0.0372 | Acc: 98.66%

Epoch 10/10:
Train Loss: 0.0380 | Acc: 98.49%
Val Loss: 0.0387 | Acc: 98.18%

Test Accuracy: 87.82%

Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.98      0.69      0.81       234
   PNEUMONIA       0.84      0.99      0.91       390

    accuracy      

In [ ]:
#import کتابخانه های لازم
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit


#تعریف هایپرپارامترها
BATCH_SIZE = 32
DROPOUT = 0.5
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
IMG_SIZE = 224
PATIENCE = 5
N_FEATURES = 100

#data augmentation
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#بخش بارگیری داده ها
def setup_dataset():
    base_path = '/content/chest_xray'
    if not os.path.exists(base_path):
        print("Downloading dataset...")
        os.makedirs('/content', exist_ok=True)
        os.system('kaggle datasets download -d paultimothymooney/chest-xray-pneumonia')
        os.system('unzip -q chest-xray-pneumonia.zip')

    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')
    return train_path, test_path

train_path, test_path = setup_dataset()

#بخش های آموزشی دیتا ست
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=val_transform)


targets = np.array([label for _, label in train_dataset])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)

#محاسبه وزن کلاس ها برای handling imbalance
class_counts = np.bincount(targets[train_idx])
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights, dtype=torch.float32)

#ایجاد بخش های Dataloader
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

#تعریف مدل شبکه عصبی
def create_model():
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(DROPOUT),
        nn.Linear(num_ftrs, 2))
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#تابع ارزیابی مدل برای gwo
def evaluate_model(feature_mask):
    model = create_model().to(device)


    for name, param in model.named_parameters():
        if 'fc' in name and 'weight' in name:

            mask = torch.tensor(feature_mask, dtype=torch.float32).to(device).view(1, -1)
            param.data = param.data * mask

    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)


    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        break


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

#تابع هدف برای الگوریتم gwo
def objective_function(position):

    feature_mask = (position > 0.5).astype(float)


    if np.sum(feature_mask) == 0:
        return 100


    accuracy = evaluate_model(feature_mask)


    error = 1.0 - accuracy


    penalty = 0.001 * np.sum(feature_mask)

    return error + penalty

#پیاده سازی الگوریتم گرگ خاکستری
class GWO:
    def __init__(self, obj_function, dim, n_agents, max_iter, lb, ub):
        self.obj_function = obj_function
        self.dim = dim
        self.n_agents = n_agents
        self.max_iter = max_iter
        self.lb = lb
        self.ub = ub


        self.positions = np.random.uniform(lb, ub, (n_agents, dim))


        self.alpha_pos = np.zeros(dim)
        self.alpha_score = float("inf")

        self.beta_pos = np.zeros(dim)
        self.beta_score = float("inf")

        self.delta_pos = np.zeros(dim)
        self.delta_score = float("inf")


        self.best_scores = []

    def optimize(self):
        for iter in range(self.max_iter):
            for i in range(self.n_agents):

                self.positions[i, :] = np.clip(self.positions[i, :], self.lb, self.ub)


                fitness = self.obj_function(self.positions[i, :])


                if fitness < self.alpha_score:
                    self.alpha_score = fitness
                    self.alpha_pos = self.positions[i, :].copy()
                elif fitness < self.beta_score:
                    self.beta_score = fitness
                    self.beta_pos = self.positions[i, :].copy()
                elif fitness < self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = self.positions[i, :].copy()


            a = 2 - iter * (2 / self.max_iter)


            for i in range(self.n_agents):
                for j in range(self.dim):

                    A1 = 2 * a * np.random.random() - a
                    C1 = 2 * np.random.random()

                    A2 = 2 * a * np.random.random() - a
                    C2 = 2 * np.random.random()

                    A3 = 2 * a * np.random.random() - a
                    C3 = 2 * np.random.random()


                    D_alpha = abs(C1 * self.alpha_pos[j] - self.positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    D_beta = abs(C2 * self.beta_pos[j] - self.positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    D_delta = abs(C3 * self.delta_pos[j] - self.positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta


                    self.positions[i, j] = (X1 + X2 + X3) / 3


            self.best_scores.append(self.alpha_score)


            print(f"Iteration {iter+1}/{self.max_iter}, Best Error: {self.alpha_score:.4f}, "
                  f"Best Accuracy: {1 - self.alpha_score:.4f}")

        return self.alpha_pos, 1 - self.alpha_score

#اجرای الگوریتم گرگ خاکستری برای انتخاب ویژگی
dim = 512
n_agents = 10
max_iter = 20
lb = 0
ub = 1

print("Starting GWO optimization...")
gwo = GWO(objective_function, dim, n_agents, max_iter, lb, ub)
best_position, best_accuracy = gwo.optimize()

#آموزش مدل نهایی با ویژگی های انتخاب شده
best_feature_mask = (best_position > 0.5).astype(float)
print(f"\nSelected {np.sum(best_feature_mask)} features out of {dim}")


print("\nTraining final model with selected features...")
final_model = create_model().to(device)


for name, param in final_model.named_parameters():
    if 'fc' in name and 'weight' in name:
        mask = torch.tensor(best_feature_mask, dtype=torch.float32).to(device).view(1, -1)
        param.data = param.data * mask


criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(final_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2)

best_val_accuracy = 0
no_improve = 0

for epoch in range(NUM_EPOCHS):
    final_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total


    final_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = final_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    scheduler.step(val_acc)

    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%')

    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(final_model.state_dict(), 'best_model.pth')
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f'\nEarly stopping at epoch {epoch+1}')
            break

#ارزیابی مدل روی داده های تست
final_model.load_state_dict(torch.load('best_model.pth'))
final_model.eval()

all_labels = []
all_preds = []
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
#ارزیابی نهایی مدل
test_acc = 100 * test_correct / len(test_dataset)
print(f'\nFinal Test Accuracy: {test_acc:.2f}%')
print('\nClassification Report:')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
print('\nConfusion Matrix:')
print(confusion_matrix(all_labels, all_preds))

Starting GWO optimization...
Iteration 1/20, Best Error: 0.3920, Best Accuracy: 0.6080
Iteration 2/20, Best Error: 0.3636, Best Accuracy: 0.6364
Iteration 3/20, Best Error: 0.3636, Best Accuracy: 0.6364
Iteration 4/20, Best Error: 0.3521, Best Accuracy: 0.6479
Iteration 5/20, Best Error: 0.3521, Best Accuracy: 0.6479
Iteration 6/20, Best Error: 0.3046, Best Accuracy: 0.6954
Iteration 7/20, Best Error: 0.3046, Best Accuracy: 0.6954
Iteration 8/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 9/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 10/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 11/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 12/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 13/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 14/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 15/20, Best Error: 0.3004, Best Accuracy: 0.6996
Iteration 16/20, Best Error: 0.2715, Best Accuracy: 0.7285
Iteration 17/20, Best Error: 0.2715,